In [28]:
import numpy as np
import cv2
import math
from scipy import ndimage
import pytesseract
from tqdm import tqdm_notebook
import pandas as pd
from ipywidgets import IntProgress

In [2]:
IMAGE_FILE_LOCATION = "C:/Users/zhiqian/Downloads/coordinates.png" 
VIDEO_FILE_LOCATION = "C:/Users/zhiqian/Downloads/videotest.mp4" 

input_img = cv2.imread(IMAGE_FILE_LOCATION) # image read

In [3]:
pytesseract.pytesseract.tesseract_cmd = r'C:\Users\zhiqian\AppData\Local\Tesseract-OCR\tesseract.exe'

custom_oem_psm_config_1 = r'--psm 10 --oem 3 -c tessedit_char_whitelist=1234'
custom_oem_psm_config_2 = r'--psm 6 --oem 3 -c tessedit_char_whitelist=0123456789:'
custom_oem_psm_config_3 = r'--psm 6 --oem 3 -c tessedit_char_whitelist=0123456789'

In [4]:
# coordinates = [(1002, 622), (1220, 652)]
coordinates = [(1002, 662), (1220, 689)]
coordinates_1 = [(1022, 662), (1076, 689)]
coordinates_2 = [(1078, 662), (1153, 689)]
coordinates_2_min = [(1083, 662), (1110, 688)]
coordinates_2_sec = [(1118, 662), (1153, 689)]

coordinates_3 = [(1159, 662), (1201, 689)]

In [19]:
1220-1002

218

In [40]:
def search_box(frame):
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
 
    ret,thresh = cv2.threshold(gray,127,255,1)
    contours,h = cv2.findContours(thresh,1,2)

    for cnt in contours:
        (x,y,w,h) = cv2.boundingRect(cnt)

        if x>1018 & y>540 & w>100 & h>25:
            approx = cv2.approxPolyDP(cnt,0.01*cv2.arcLength(cnt,True),True)
#             print(x, y, w, h, len(approx))

            if len(approx)==4:
                cv2.drawContours(img,[cnt],0,(0,0,255),-1)
                cv2.rectangle(img, (x,y), (x+w,y+h), (0,255,0), 2)
                
                cv2.imwrite('rect.png',img)
                print('FOund')

In [41]:
# Test box search
cap = cv2.VideoCapture(VIDEO_FILE_LOCATION)
max_count = cap.get(cv2.CAP_PROP_FRAME_COUNT)

# Progress Bar
f = IntProgress(min=0, max=max_count) # instantiate the bar
display(f) # display the bar

# Check if camera opened successfully
if (cap.isOpened()== False): 
    print("Error opening video stream or file")

count = 0
# Read until video is completed
while(cap.isOpened()):
    # Capture frame-by-frame
    ret, frame = cap.read()

    if ret == True:
        temp = []
        
        search_box(frame)
        
        count += 30 # i.e. at 30 fps, this advances one second
        f.value = count # signal to increment the progress bar
        cap.set(1, count)
        
        
        # Press Q on keyboard to  exit
        if cv2.waitKey(25) & 0xFF == ord('q'):
            break
            
    # Break the loop
    else: 
        break
        
# When everything done, release the video capture object
cap.release()
 
# Closes all the frames
cv2.destroyAllWindows()

IntProgress(value=0, max=18056)

In [34]:
box_length = coordinates[1][0] - coordinates[0][0]

In [38]:
ratio_1 = (coordinates_1[1][0] - coordinates_1[0][0])/box_length
ratio_2 = (coordinates_2[1][0] - coordinates_2[0][0])/box_length
ratio_3 = (coordinates_3[1][0] - coordinates_3[0][0])/box_length

print(ratio_1, ratio_2, ratio_3)

0.24770642201834864 0.3440366972477064 0.1926605504587156


In [24]:
def get_time_from_frame(frame, coordinates, config):
    image_roi = frame[coordinates[0][1]:coordinates[1][1], 
                           coordinates[0][0]:coordinates[1][0]] 

    text = pytesseract.image_to_string(image_roi, config=config)
    return text

In [30]:
total_list = []

In [31]:
cap = cv2.VideoCapture(VIDEO_FILE_LOCATION)
# fps = cap.get(cv2.CAP_PROP_FPS)

max_count = cap.get(cv2CAP_PROP_FRAME_COUNT)

# Progress Bar
f = IntProgress(min=0, max=max_count) # instantiate the bar
display(f) # display the bar

# Check if camera opened successfully
if (cap.isOpened()== False): 
    print("Error opening video stream or file")

count = 0
# Read until video is completed
while(cap.isOpened()):
    # Capture frame-by-frame
    ret, frame = cap.read()

    if ret == True:
        temp = []
        
        # Get timestamp
        timestamp = cap.get(cv2.CAP_PROP_POS_MSEC)
        
        # Drawing a rectangle around the region of interest (roi)
#         cv2.rectangle(frame, coordinates_2_min[0], coordinates_2_min[1], (0,0,255), 2) 

        # Get in game time info
        quarter = get_time_from_frame(frame, coordinates=coordinates_1, config=custom_oem_psm_config_1)
        time = get_time_from_frame(frame, coordinates=coordinates_2, config=custom_oem_psm_config_2)
        clock = get_time_from_frame(frame, coordinates=coordinates_3, config=custom_oem_psm_config_3)

        # Display the resulting frame
#         cv2.imshow('Frame',frame)

        # Append to total list
        total_list.append([timestamp, quarter, time, clock])


        count += 30 # i.e. at 30 fps, this advances one second
        f.value = count # signal to increment the progress bar
        cap.set(1, count)
        

        # Press Q on keyboard to  exit
        if cv2.waitKey(25) & 0xFF == ord('q'):
            break
            
    # Break the loop
    else: 
        break
        
# When everything done, release the video capture object
cap.release()
 
# Closes all the frames
cv2.destroyAllWindows()

IntProgress(value=0, max=18056)

NameError: name 'quarter' is not defined

In [27]:
# Convert list into DF
df = pd.DataFrame(total_list, columns=['Timestamp','Quarter','Gametime','Clock'])
df['Timestamp'] = df['Timestamp']/1000

In [28]:
df.to_csv('timestamp2.csv')

## Appendix

In [15]:
# # REGION OF INTEREST (ROI) SELECTION

# # initializing the list for storing the coordinates 
# coordinates = [] 
  
# # Defining the event listener (callback function)
# def shape_selection(event, x, y, flags, param): 
#     # making coordinates global
#     global coordinates 
  
#     # Storing the (x1,y1) coordinates when left mouse button is pressed  
#     if event == cv2.EVENT_LBUTTONDOWN: 
#         coordinates = [(x, y)] 
  
#     # Storing the (x2,y2) coordinates when the left mouse button is released and make a rectangle on the selected region
#     elif event == cv2.EVENT_LBUTTONUP: 
#         coordinates.append((x, y)) 
#         print(coordinates)
#         # Drawing a rectangle around the region of interest (roi)
#         cv2.rectangle(image, coordinates[0], coordinates[1], (0,0,255), 2) 
#         cv2.imshow("image", image) 

# # load the image, clone it, and setup the mouse callback function 
# image = input_img
# image_copy = image.copy()
# cv2.namedWindow("image") 
# cv2.setMouseCallback("image", shape_selection) 
  
# # keep looping until the 'q' key is pressed 
# while True: 
#     # display the image and wait for a keypress 
#     cv2.imshow("image", image) 
#     key = cv2.waitKey(1) & 0xFF
  
#     if key==13: # If 'enter' is pressed, apply OCR
#         break
    
#     if key == ord("c"): # Clear the selection when 'c' is pressed 
#         image = image_copy.copy() 

# # if len(coordinates) == 2: 
# #     image_roi = image_copy[coordinates[0][1]:coordinates[1][1], 
# #                                coordinates[0][0]:coordinates[1][0]] 
# #     cv2.imshow("Selected Region of Interest - Press any key to proceed", image_roi) 
# #     cv2.waitKey(0) 
    
# # # closing all open windows 
# # cv2.destroyAllWindows()  